# UB TUBAF MAB2 Datenanalyse

Dieses Notebook analysiert die MAB2-Daten der Universitätsbibliothek TU Bergakademie Freiberg.

## 🎯 Ziel
- Parse MAB2-Format (027out.t) 
- Extrahiere bibliographische Daten
- Vorbereitung für Bestandsvergleich mit VDEH

## 📚 Datenquelle
- **Datei**: `027out.t` (MAB2-Binärformat)
- **Quelle**: UB TU Bergakademie Freiberg Bestandsdaten
- **Format**: MAB2 (Maschinelles Austauschformat für Bibliotheken)

In [1]:
# 🛠️ SETUP: Konfiguration und Module laden
import sys
from pathlib import Path

📁 Projektroot: /media/sz/Data/Bibo/analysis
🔧 Python-Pfad erweitert: /media/sz/Data/Bibo/analysis/src


In [2]:
# 📋 KONFIGURATION LADEN
try:
    from config_loader import load_config
    from pathlib import Path
    
    config = load_config(project_root / 'config.yaml')
    print("✅ Konfiguration geladen")
    
    # UB TUBAF spezifische Konfiguration und Pfade
    ub_config = config.get('data_sources.ub_tubaf', {})
    
    if not ub_config:
        raise ValueError("UB TUBAF Konfiguration nicht gefunden!")
    
    # Stelle sicher, dass alle benötigten Verzeichnisse existieren
    required_dirs = [
        config.get_path('data.root'),
        config.get_path('data.ub_tubaf.raw'),
        config.get_path('data.ub_tubaf.processed'),
        config.get_path('data.ub_tubaf.exports')
    ]
    
    for dir_path in required_dirs:
        if not dir_path.exists():
            print(f"📂 Erstelle Verzeichnis: {dir_path}")
            dir_path.mkdir(parents=True, exist_ok=True)
    
    # Überprüfe den Datenpfad
    data_file = project_root / ub_config.get('path', '')
    if not data_file.exists():
        raise FileNotFoundError(f"Datendatei nicht gefunden: {data_file}")
        
    print(f"📄 UB TUBAF Datenquelle: {ub_config.get('description', 'Keine Beschreibung')}")
    print(f"📂 Dateipfad: {data_file}")
    print(f"🔤 Encoding: {ub_config.get('encoding', 'utf-8')}")
    print(f"📊 Geschätzte Records: {ub_config.get('estimated_records', 'Unbekannt')}")

except ImportError as e:
    print(f"❌ Fehler beim Laden des config_loader Moduls: {e}")
    raise
except ValueError as e:
    print(f"❌ Fehler in der Konfiguration: {e}")
    raise
except FileNotFoundError as e:
    print(f"❌ Datei nicht gefunden: {e}")
    raise
except Exception as e:
    print(f"❌ Unerwarteter Fehler: {e}")
    raise

✅ Konfiguration geladen: /media/sz/Data/Bibo/analysis/config.yaml
✅ Konfiguration geladen
📄 UB TUBAF Datenquelle: Bestandsdaten der UB TU Bergakademie Freiberg
📂 Dateipfad: /media/sz/Data/Bibo/analysis/data/ub_tubaf/raw/027out.t
🔤 Encoding: latin1
📊 Geschätzte Records: None


In [3]:
# 📦 BIBLIOTHEKEN UND PARSER LADEN
import pandas as pd
import matplotlib.pyplot as plt

# MAB2 Parser importieren
from parsers.mab2_parser import MAB2Parser, analyze_mab2_data, get_sample_records_mab2

print("✅ Bibliotheken und Parser geladen")
print("📚 MAB2Parser bereit für UB TUBAF Daten")

✅ Bibliotheken und Parser geladen
📚 MAB2Parser bereit für UB TUBAF Daten


In [4]:
# 📑 MAB2 BEISPIELDATEN
content = """
### 00499nM2.01200024      h
001 073871532
002a20020501
003 20180317150015
026k073871532
026 GBV073871532
030 e|5z|||z|0035
050 a|||||||||||||
051 n||||||
060 $akartografisches Bild$bcri
061 $aohne Hilfsmittel zu benutzen$bn
062 $aBlatt$bnb
070aBSZ
076bmb
200bBayern / Geologisches Landesamt
310 Geologische Karte von Bayern <1:25000>
331 Geologische Karte von Bayern
335 Bundesrepublik Deutschland
359 hrsg. vom Bayerischen Geologischen Landesamt
407 1 : 25 000
410 München
433 Je Bl. 47,5 x 44,5 cm

### 00599nM2.01200024      u
001 9073875056
002a20020501
003 20230321140005
010 073871532
013k073871532
026k073875058
026 GBV073875058
030 e|5z|||z|00||
050 a|||||||||||||
051 |||||||
060 $akartografisches Bild$bcri
061 $aohne Hilfsmittel zu benutzen$bn
062 $aBlatt$bnb
070aBSZ
076bod
076dkart
081 Geologische Karte von Bayern <1:25000>
081aLinderhof ; 1
089 8431, 1
090 8431.1967
100bKuhnert, Christian ¬[Bearb.]¬
200bBayern / Geologisches Landesamt
331 [Karte]
359 bearb. von Ch. Kuhnert
370aGeologische Karte von Bayern <1:25000>
370aLinderhof
407 1 : 25 000
410 München
425 1967
425a1967
433 1 Kt

### 00481nM2.01200024      h
001 9125700332
002a19930613
003 20230519125958
026k125700334
026 GBV125700334
030 a|5z|||z|0037
036aXA-DDDE
037bger
050 a|||||||||||||
051 n||||||
060 $aText$btxt
061 $aohne Hilfsmittel zu benutzen$bn
062 $aBand$bnc
070aBSZ
076bmb
200bDeutschland <DDR> / Amt für Standardisierung, Messwesen und Warenprüfung
202b191578665
331 Verzeichnis staatlicher Standards der DDR
410 Berlin
412 Verl. für Standardisierung
425 1986-
425a1986
540aISBN 3-7405-0071-9

### 00718nM2.01200024      u
001 9162859218
002a19940909
003 20230519130014
010 9125700332
013k125700334
013s9125700332
026k16285921X
026 GBV16285921X
030 e|5z|||z|00||
036aXA-DDDE
037bger
050 a|||||||||||||
051 |||||||
060 $aText$btxt
061 $aohne Hilfsmittel zu benutzen$bn
062 $aBand$bnc
064a$aVerzeichnis$9210051078
070aBSZ
076bod
081 Verzeichnis staatlicher Standards der DDR
089 1
090 1.1989
200bDeutschland <DDR> / Amt für Standardisierung, Messwesen und Warenprüfung
202b191578665
331 Sachteil (2.0.) - (662.0)
410 Berlin
412 Verl. für Standardisierung
425 1989
425a1989
433 550 S.
540aISBN 3-7405-0072-7
902g  208896155           Deutschland
902s  212249827           Norm
902s  210164832           TGL
904aSWB
"""

print("✅ MAB2 Beispieldaten geladen")

✅ MAB2 Beispieldaten geladen


In [5]:
# 🔍 MAB2 Parser Implementierung
import re
import pandas as pd
from typing import Optional, Dict, List

class SimpleMab2Parser:
    """Ein einfacher Parser für MAB2-Datensätze"""
    
    def __init__(self):
        # Wichtige MAB2-Felder für die Extraktion
        self.key_fields = {
            '001': 'id',
            '100': 'author',
            '200': 'institution',
            '331': 'title',
            '425': 'year',
            '410': 'place',
            '412': 'publisher',
            '433': 'physical_desc',
            '540': 'isbn'
        }
        
        # Statistiken
        self.stats = {
            'total_records': 0,
            'parsed_records': 0,
            'field_counts': {},
            'field_quality': {}  # Neue Statistik für Datenqualität
        }
    
    def parse_content(self, content: str) -> pd.DataFrame:
        """Parst MAB2-Content und gibt DataFrame zurück"""
        # Teile Content in Records
        records = []
        raw_records = content.split('###')
        
        for raw_record in raw_records:
            if raw_record.strip():
                record = self._parse_record(raw_record)
                if record:
                    records.append(record)
                    
        # Erstelle DataFrame
        df = pd.DataFrame(records)
        
        # Nachbearbeitung der Felder
        df = self._post_process_fields(df)
        
        # Aktualisiere Statistiken
        self.stats['total_records'] = len(raw_records) - 1  # -1 wegen Split
        self.stats['parsed_records'] = len(records)
        self._update_field_quality_stats(df)
        
        return df
    
    def _clean_content(self, content: str) -> str:
        """Bereinigt Feldinhalt von MAB2-spezifischen Markierungen"""
        if not content:
            return ""
            
        # Entferne Subfeldmarkierungen
        content = re.sub(r'\$[a-z]', ' ', content)
        # Entferne Bearbeitungsklammern
        content = re.sub(r'¬\[.*?\]¬', '', content)
        # Bereinige spezielle Zeichen
        content = re.sub(r'[<>{}\\]', '', content)
        # Bereinige mehrfache Leerzeichen
        content = ' '.join(content.split())
        return content.strip()
    
    def _extract_year(self, year_str) -> Optional[str]:
        """Extrahiert das Jahr aus verschiedenen Formaten"""
        if pd.isna(year_str):
            return None
            
        # Konvertiere zu String wenn nötig
        year_str = str(year_str)
        
        # Verschiedene Jahr-Patterns
        patterns = [
            r'(\d{4})',           # Standard Jahr
            r'(\d{4})[\-\.]',     # Jahr mit Suffix
            r'©(\d{4})',          # Copyright Jahr
            r'\[(\d{4})\]'        # Jahr in Klammern
        ]
        
        for pattern in patterns:
            match = re.search(pattern, year_str)
            if match:
                year = match.group(1)
                # Validiere Jahr (1400-2100)
                if 1400 <= int(year) <= 2100:
                    return year
        return None
    
    def _clean_author(self, author) -> Optional[str]:
        """Bereinigt und normalisiert Autorennamen"""
        if pd.isna(author):
            return None
            
        # Konvertiere zu String wenn nötig
        author = str(author)
        if not author:
            return None
            
        # Entferne Rollenbezeichnungen
        author = re.sub(r'\[.*?\]', '', author)
        # Entferne Lebensdaten
        author = re.sub(r'\(.*?\)', '', author)
        # Entferne eckige Klammern und deren Inhalt (oft Funktionsbezeichnungen)
        author = re.sub(r'¬.*?¬', '', author)
        # Bereinige und normalisiere
        author = self._clean_content(author)
        return author if author else None
    
    def _clean_isbn(self, isbn) -> Optional[str]:
        """Bereinigt und validiert ISBN"""
        if pd.isna(isbn):
            return None
            
        # Konvertiere zu String wenn nötig
        isbn = str(isbn)
        if not isbn:
            return None
            
        # Extrahiere ISBN
        match = re.search(r'(?:ISBN[- ]*)?([\dX\-]+)', isbn)
        if match:
            # Entferne Bindestriche
            isbn = re.sub(r'[^\dX]', '', match.group(1))
            # Validiere Länge (ISBN-10 oder ISBN-13)
            if len(isbn) in [10, 13]:
                return isbn
        return None
    
    def _parse_record(self, raw_record: str) -> dict:
        """Parst einen einzelnen MAB2-Record"""
        record_data = {}
        
        # Verarbeite jede Zeile
        for line in raw_record.splitlines():
            line = line.strip()
            if not line:
                continue
            
            # Extrahiere Feldcode und Inhalt
            if len(line) >= 3 and line[:3].rstrip('abcdefghijklmnopqrstuvwxyz').isdigit():
                field_code = line[:3]
                content = line[3:].strip()
                
                # Extrahiere den eigentlichen Inhalt
                if content and content[0] in 'abcdefghijklmnopqrstuvwxyz':
                    content = content[1:]  # Entferne Subfeldkennung
                
                # Bereinige Inhalt
                content = self._clean_content(content)
                
                # Speichere Feld wenn es in key_fields ist
                if field_code in self.key_fields:
                    field_name = self.key_fields[field_code]
                    record_data[field_name] = content
                
                # Zähle Feldvorkommen für Statistik
                if field_code not in self.stats['field_counts']:
                    self.stats['field_counts'][field_code] = 0
                self.stats['field_counts'][field_code] += 1
        
        return record_data
    
    def _post_process_fields(self, df: pd.DataFrame) -> pd.DataFrame:
        """Nachbearbeitung der geparsten Felder"""
        # Jahr
        if 'year' in df.columns:
            df['year'] = df['year'].apply(self._extract_year)
            
        # ISBN
        if 'isbn' in df.columns:
            df['isbn'] = df['isbn'].apply(self._clean_isbn)
            
        # Autor
        if 'author' in df.columns:
            df['author'] = df['author'].apply(self._clean_author)
            
        # Titel
        if 'title' in df.columns:
            df['title'] = df['title'].apply(lambda x: self._clean_content(x) if pd.notna(x) else None)
            
        # Institution
        if 'institution' in df.columns:
            df['institution'] = df['institution'].apply(lambda x: self._clean_content(x) if pd.notna(x) else None)
            
        # Verlag
        if 'publisher' in df.columns:
            df['publisher'] = df['publisher'].apply(lambda x: self._clean_content(x) if pd.notna(x) else None)
            
        # Physische Beschreibung
        if 'physical_desc' in df.columns:
            df['physical_desc'] = df['physical_desc'].apply(lambda x: self._clean_content(x) if pd.notna(x) else None)
            
        return df
    
    def _update_field_quality_stats(self, df: pd.DataFrame) -> None:
        """Aktualisiert Statistiken zur Datenqualität"""
        total_records = len(df)
        self.stats['field_quality'] = {}
        
        for col in df.columns:
            non_null = df[col].count()
            quality = {
                'vorhanden': non_null,
                'fehlend': total_records - non_null,
                'abdeckung': f"{(non_null/total_records*100):.1f}%"
            }
            if non_null > 0:
                unique = df[col].nunique()
                quality['eindeutige_werte'] = unique
                quality['eindeutigkeit'] = f"{(unique/non_null*100):.1f}%"
            
            self.stats['field_quality'][col] = quality
    
    def get_stats(self) -> dict:
        """Gibt erweiterte Parsing-Statistiken zurück"""
        stats = {
            'Gesamtrecords': self.stats['total_records'],
            'Erfolgreich geparst': self.stats['parsed_records'],
            'Erfolgsrate': f"{(self.stats['parsed_records']/max(1, self.stats['total_records'])*100):.1f}%",
            'Top 10 Felder': dict(sorted(
                self.stats['field_counts'].items(), 
                key=lambda x: x[1], 
                reverse=True
            )[:10]),
            'Datenqualität': self.stats['field_quality']
        }
        return stats

# Erstelle Parser-Instanz
parser = SimpleMab2Parser()

# Parse Beispieldaten
print("🔍 Parse MAB2 Beispieldaten...")
df = parser.parse_content(content)

# Zeige Ergebnisse
print(f"\n✅ Parsing abgeschlossen!")
print(f"📊 Records im DataFrame: {len(df)}")

print("\n📋 Spalten im DataFrame:")
for col in df.columns:
    non_null = df[col].count()
    pct = non_null/len(df)*100
    print(f"   {col:15}: {non_null:3d} Werte ({pct:6.1f}%)")

print("\n📈 Parser Statistiken:")
stats = parser.get_stats()
for key, value in stats.items():
    print(f"\n   {key}:")
    if isinstance(value, dict):
        if key == 'Datenqualität':
            for field, quality in value.items():
                print(f"\n      {field}:")
                for metric, val in quality.items():
                    print(f"         {metric}: {val}")
        else:
            for k, v in value.items():
                print(f"      {k}: {v}")
    else:
        print(f"      {value}")

print("\n🔍 Beispiel-Records:")
print("\nErster Record:")
print(df.iloc[0].to_string())
print("\nZweiter Record:")
print(df.iloc[1].to_string())

🔍 Parse MAB2 Beispieldaten...

✅ Parsing abgeschlossen!
📊 Records im DataFrame: 4

📋 Spalten im DataFrame:
   id             :   4 Werte ( 100.0%)
   institution    :   4 Werte ( 100.0%)
   title          :   4 Werte ( 100.0%)
   place          :   4 Werte ( 100.0%)
   physical_desc  :   3 Werte (  75.0%)
   author         :   1 Werte (  25.0%)
   year           :   3 Werte (  75.0%)
   publisher      :   2 Werte (  50.0%)
   isbn           :   2 Werte (  50.0%)

📈 Parser Statistiken:

   Gesamtrecords:
      4

   Erfolgreich geparst:
      4

   Erfolgsrate:
      100.0%

   Top 10 Felder:
      026: 8
      425: 6
      076: 5
      001: 4
      002: 4
      003: 4
      030: 4
      050: 4
      051: 4
      060: 4

   Datenqualität:

      id:
         vorhanden: 4
         fehlend: 0
         abdeckung: 100.0%
         eindeutige_werte: 4
         eindeutigkeit: 100.0%

      institution:
         vorhanden: 4
         fehlend: 0
         abdeckung: 100.0%
         eindeutige_wer

# 📚 MAB2 Parser Details

Der neue Parser wurde gezielt für die MAB2-Daten der UB TUBAF entwickelt und fokussiert sich auf die wichtigsten bibliographischen Felder.

## 🎯 Funktionsweise

1. **Record-Trennung**:
   - Datensätze werden durch `###` getrennt
   - Jeder Record enthält mehrere Felder mit 3-stelligen Codes

2. **Feldverarbeitung**:
   - Codes wie `001`, `331`, `425` identifizieren Felder
   - Subfelder (z.B. `$a`, `$b`) werden automatisch bereinigt
   - Spezielle Kennungen werden normalisiert

3. **Extraktion wichtiger Felder**:
   - `001`: Identifikationsnummer
   - `100`: Autor
   - `200`: Institution/Körperschaft
   - `331`: Titel
   - `425`: Erscheinungsjahr
   - `410`: Erscheinungsort
   - `412`: Verlag
   - `433`: Physische Beschreibung
   - `540`: ISBN

## 📊 Statistiken
- Anzahl verarbeiteter Records
- Erfolgsrate des Parsings
- Häufigkeit der einzelnen Felder

In [6]:
# 🧪 MAB2 Parser Testen

# Importiere den verbesserten Parser
from parsers.mab2_parser import MAB2Parser, analyze_mab2_data, get_sample_records_mab2
import logging

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format='%(message)s')

# 1️⃣ Test mit Beispieldaten
print("🔍 TESTE BEISPIELDATEN")
print("-" * 50)

# Lese die Beispieldaten als Datei ein
import tempfile
from pathlib import Path

# Erstelle temporäre Datei mit den Beispieldaten
with tempfile.NamedTemporaryFile(mode='w', suffix='.dat', encoding='latin1', delete=False) as temp:
    temp.write(content)
    temp_path = Path(temp.name)

try:
    # Parse die temporäre Datei
    parser = MAB2Parser(encoding='latin1')
    df_example = parser.parse_file(temp_path)

    print("\n📊 Analyse der Beispieldaten:")
    analyze_mab2_data(df_example)

    # 2️⃣ Test mit vollständigem Datenbestand
    print("\n\n🔍 TESTE GESAMTEN DATENBESTAND")
    print("-" * 50)

    # Lade MAB2-Datei aus der Konfiguration
    data_file = config.get_path('data.ub_tubaf.raw') / '027out.t'
    
    if not data_file.exists():
        print(f"❌ Datei nicht gefunden: {data_file}")
    else:
        print(f"📂 Verarbeite Datei: {data_file}")
        print(f"📊 Encoding: {ub_config.get('encoding', 'latin1')}")
        
        # Parse Daten
        parser = MAB2Parser(encoding=ub_config.get('encoding', 'latin1'))
        df_full = parser.parse_file(data_file)
        
        print("\n📊 Analyse des Gesamtdatenbestands:")
        analyze_mab2_data(df_full)
        
        # Zeige Beispielrecords
        print("\n🔍 Beispiel-Records aus dem Gesamtdatenbestand:")
        sample_df = get_sample_records_mab2(df_full, n=3)
        print(sample_df.to_string())
        
except Exception as e:
    print(f"❌ Fehler beim Verarbeiten der Daten: {str(e)}")
finally:
    # Lösche die temporäre Datei
    if 'temp_path' in locals():
        temp_path.unlink()

🔄 Lade MAB2-Datei: /tmp/tmpkyoepxa9.dat
📊 Encoding: latin1
📋 Gefundene Records: 4

✅ ERFOLGREICH GEPARST (Record 1):
📊 Encoding: latin1
📋 Gefundene Records: 4

✅ ERFOLGREICH GEPARST (Record 1):
   id: 00499
   source: ub_tubaf_mab2
   title: Geologische Karte von Bayern <1:25000> : Geologische Karte von Bayern
   authors: ['Bayern / Geologisches Landesamt']
   authors_str: Bayern / Geologisches Landesamt
   year: None
   isbn: None
   place: München
   physical_desc: Je Bl. 47,5 x 44,5 cm
   id: 00499
   source: ub_tubaf_mab2
   title: Geologische Karte von Bayern <1:25000> : Geologische Karte von Bayern
   authors: ['Bayern / Geologisches Landesamt']
   authors_str: Bayern / Geologisches Landesamt
   year: None
   isbn: None
   place: München
   physical_desc: Je Bl. 47,5 x 44,5 cm
   title_candidates: ['Geologische Karte von Bayern <1:25000>', 'Geologische Karte von Bayern', 'hrsg. vom Bayerischen Geologischen Landesamt']
❌ Fehler beim Parsen von Record 2:
   expected string or bytes

🔍 TESTE BEISPIELDATEN
--------------------------------------------------

📊 Analyse der Beispieldaten:


🔍 TESTE GESAMTEN DATENBESTAND
--------------------------------------------------
📂 Verarbeite Datei: /media/sz/Data/Bibo/analysis/data/ub_tubaf/raw/027out.t
📊 Encoding: latin1


📋 Gefundene Records: 518,954

✅ ERFOLGREICH GEPARST (Record 1):
   id: 00499
   source: ub_tubaf_mab2
   title: Geologische Karte von Bayern <1:25000> : Geologische Karte von Bayern
   authors: ['Bayern / Geologisches Landesamt']
   authors_str: Bayern / Geologisches Landesamt
   year: None
   isbn: None
   place: München
   physical_desc: Je Bl. 47,5 x 44,5 cm
   title_candidates: ['Geologische Karte von Bayern <1:25000>', 'Geologische Karte von Bayern', 'hrsg. vom Bayerischen Geologischen Landesamt']
❌ Fehler beim Parsen von Record 2:
   expected string or bytes-like object, got 'list'
   Record-Start: ### 00599nM2.01200024      u
001 9073875056
002a20020501
003 20230321140005
010 073871532
013k073871...
❌ Fehler beim Parsen von Record 3:
   expected string or bytes-like object, got 'list'
   Record-Start: ### 00481nM2.01200024      h
001 9125700332
002a19930613
003 20230519125958
026k125700334
026 GBV125...
❌ Fehler beim Parsen von Record 4:
   expected string or bytes-like object, 


📊 Analyse des Gesamtdatenbestands:

🔍 Beispiel-Records aus dem Gesamtdatenbestand:
      id                                                  title                      authors_str    year  isbn    place
0  00499  Geologische Karte von Bayern <1:25000> : Geologisc...  Bayern / Geologisches Landesamt     NaN  None  München
1  00379                           Caesar's wife : Patrick Gale                    Gale, Patrick     NaN  None     None
2  00919                         Medienbildung und Gesellschaft                             None  2006.0  None     None
